In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv('D:\Datasetss\Movie-Dataset-Latest.csv')

In [32]:
df.head()

,Unnamed: 0,id,title,release_date,overview,popularity,vote_average,vote_count,video
0,0,19404,Dilwale Dulhania Le Jayenge,1995-10-20,"Raj is a rich, carefree, happy-go-lucky second...",25.884,8.7,3304,False
1,1,278,The Shawshank Redemption,1994-09-23,Framed in the 1940s for the double murder of h...,60.110,8.7,20369,False
2,2,238,The Godfather,1972-03-14,"Spanning the years 1945 to 1955, a chronicle o...",62.784,8.7,15219,False
3,3,724089,Gabriel's Inferno Part II,2020-07-31,Professor Gabriel Emerson finally learns the t...,28.316,8.6,1360,False
4,4,424,Schindler's List,1993-11-30,The true story of how businessman Oskar Schind...,38.661,8.6,12158,False


In [33]:
df.tail()

,Unnamed: 0,id,title,release_date,overview,popularity,vote_average,vote_count,video
9458,9458,13805,Disaster Movie,2008-08-29,"The filmmaking team behind the hits ""Scary Mov...",12.121,3.2,787,False
9459,9459,5491,Battlefield Earth,2000-05-12,"In the year 3000, man is no match for the Psyc...",10.200,3.2,628,False
9460,9460,11059,House of the Dead,2003-04-11,"Set on an island off the coast, a techno rave ...",8.120,3.2,281,False
9461,9461,14164,Dragonball Evolution,2009-03-12,The young warrior Son Goku sets out on a quest...,52.129,2.8,1584,False
9462,9462,40016,Birdemic: Shock and Terror,2010-02-27,A platoon of eagles and vultures attacks the r...,7.341,2.2,237,False


In [34]:
df = df[['id', 'title', 'vote_average']]


movie_rating_mean = df.groupby('id')['vote_average'].mean().reset_index()


df = pd.merge(df, movie_rating_mean, on='id', how='left')
df.rename(columns={'vote_average_x': 'vote_average', 'vote_average_y': 'mean_rating'}, inplace=True)

df['rating_norm'] = df['vote_average'] - df['mean_rating']


user_item_matrix = df.pivot_table(index='id', columns='title', values='rating_norm')


user_item_matrix.fillna(0, inplace=True)


item_similarity = cosine_similarity(user_item_matrix.T)


def get_movie_recommendations(movie_title, top_n=5):

    movie_index = df[df['title'] == movie_title].index[0]
    

    similarity_scores = list(enumerate(item_similarity[movie_index]))
    

    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    
  
    top_similar_movies = similarity_scores[1:top_n+1]
    

    movie_titles = [df.iloc[movie[0]]['title'] for movie in top_similar_movies]
    
    return movie_titles


movie_title = 'Dilwale Dulhania Le Jayenge'
recommendations = get_movie_recommendations(movie_title)
print(f"Top 5 recommendations for '{movie_title}': {recommendations}")


Top 5 recommendations for 'Dilwale Dulhania Le Jayenge': ['The Shawshank Redemption', 'The Godfather', "Gabriel's Inferno Part II", "Schindler's List", "Gabriel's Inferno"]
